In [ ]:
!pip install pysrt opencv-python moviepy
import os
import re
import cv2
import json
import pysrt
import moviepy.editor as mp
from google.colab import drive
from PIL import Image
drive.mount('/content/drive')

In [16]:
def get_episode_number(filename): # 提取數字部分作為集數
    match = re.search(r'\d+', filename)
    return match.group() if match else None

def sanitize_filename(text):  # 替換不允許的字元為全形或底線
    replace_map = {"/": "／", "\\": "＼", "?": "？", "*": "＊", "\"": "＂", "<": "＜", ">": "＞", "|": "｜", ":": "："}
    for key, value in replace_map.items():
        text = text.replace(key, value)
    return text

def read_srt(file_path):
    subs = pysrt.open(file_path)
    subtitles = []
    for index, sub in enumerate(subs, start=0):
        start_time = sub.start.to_time()
        end_time = sub.end.to_time()
        text = sub.text.replace('\n', ' ')
        subtitles.append({
            'index': index,
            'start_time': start_time,
            'end_time': end_time,
            'text': text
        })
    return subtitles

def load_video(video_path):
    try:
        return mp.VideoFileClip(video_path)
    except Exception as e:
        print(f"無法載入影片: {e}")
        return None

def capture_frames(video, subtitles, output_folder, episode, filename_prefix, image_format):
    episode_folder = os.path.join(output_folder, f"episode{int(episode):02d}")
    os.makedirs(episode_folder, exist_ok=True)
    for sub in subtitles:
        # 計算開始時間、結束時間與中間時間，包含毫秒（微秒）
        start_seconds = (sub['start_time'].hour * 3600 + sub['start_time'].minute * 60 + sub['start_time'].second + sub['start_time'].microsecond / 1e6)
        end_seconds = (sub['end_time'].hour * 3600 + sub['end_time'].minute * 60 + sub['end_time'].second + sub['end_time'].microsecond / 1e6)
        mid_time = (start_seconds + end_seconds) / 2

        # 擷取中間時間的畫面
        mid_frame = video.get_frame(mid_time)

        # 提取台詞文字與索引來作為檔案名稱的一部分，處理不允許的字元
        subtitle_text = sanitize_filename(sub['text'].replace(" ", ""))
        subtitle_index = sub['index']

        # 使用台詞文字與索引來生成檔案名稱
        mid_frame_filename = f"{filename_prefix}{int(episode):02d}_{subtitle_index:03d}_{subtitle_text}.{image_format}"
        mid_frame_path = os.path.join(episode_folder, mid_frame_filename)
        Image.fromarray(mid_frame).save(mid_frame_path)
        #cv2.imwrite(mid_frame_path, cv2.cvtColor(mid_frame, cv2.COLOR_RGB2BGR))
        print(f"已儲存: {mid_frame_path}")

def process_all_episodes(video_folder, subtitle_folder, output_base, filename_prefix, image_format):
    video_files = {get_episode_number(f): os.path.join(video_folder, f) for f in os.listdir(video_folder) if f.endswith(".mp4")}
    subtitle_files = {get_episode_number(f): os.path.join(subtitle_folder, f) for f in os.listdir(subtitle_folder) if f.endswith(".srt")}

    for episode in sorted(video_files.keys() & subtitle_files.keys()):
        video_path = video_files[episode]
        subtitle_path = subtitle_files[episode]

        print(f"處理第 {episode} 集...")
        video = load_video(video_path)
        if video:
            subtitles = read_srt(subtitle_path)
            capture_frames(video, subtitles, output_base, episode, filename_prefix, image_format)


In [ ]:
test_video_file = "/content/drive/MyDrive/SAS/mygo_video/[ANi] BanG Dream! It's MyGO!!!!! - 02 [1080P][Baha][WEB-DL][AAC AVC][CHT].mp4"
test_srt_file = "/content/drive/MyDrive/SAS/mygo_subtitle/BanG Dream! It_s MyGO!!!!! - 02.srt"

test_subtitles = read_srt(test_srt_file)
for sub in test_subtitles[:10]:
    print(sub)

test_video = load_video(test_video_file)
if test_video:
    print("影片時長:", test_video.duration, "秒")

擷取單集

In [ ]:
video_path = "/content/drive/MyDrive/SAS/ave_mujica_video/[ANi] BanG Dream! Ave Mujica - 07 [1080P][Baha][WEB-DL][AAC AVC][CHT].mp4"
subtitle_path = "/content/drive/MyDrive/SAS/ave_mujica_subtitle/BanG Dream! Ave Mujica 07.srt"
filename_prefix = "ave-mujica_ep" #mygo_ep
episode = "07"
output_base = "/content/drive/MyDrive/SAS/ave mujica screenshot"
image_format = "jpg"

video = load_video(video_path)
if video:
  subtitles = read_srt(subtitle_path)
  capture_frames(video, subtitles, output_base, episode, filename_prefix, image_format)

In [ ]:
video_folder = "/content/drive/MyDrive/SAS/ave_mujica_video"
subtitle_folder = "/content/drive/MyDrive/SAS/ave_mujica_subtitle"
output_base = "/content/drive/MyDrive/SAS/ave_mujica_screenshot"
filename_prefix = "ave_mujica_ep" #ave-mujica_ep
image_format = "jpg"

def check_folder_exists(folder_path):
    if os.path.exists(folder_path):
        print(f"Folder exists: {folder_path}")
    else:
        print(f"Folder does not exist: {folder_path}")

check_folder_exists(video_folder)
check_folder_exists(subtitle_folder)
check_folder_exists(output_base)

In [ ]:
process_all_episodes(video_folder, subtitle_folder, output_base, filename_prefix, image_format)

生成data.json

In [ ]:
# 設定圖片資料夾的路徑
image_dir = "/content/drive/MyDrive/SAS/ave_mujica_screenshot"
output_file = "/content/drive/MyDrive/SAS/ave_mujica_screenshot/data.json"

# 確保 assets 資料夾存在
os.makedirs(os.path.dirname(output_file), exist_ok=True)

# 存放圖片資訊的列表
image_data = []

# 遍歷 images 資料夾內的所有子資料夾與檔案
for root, _, files in os.walk(image_dir):
    for file in files:
        if file.lower().endswith((".jpg", ".jpeg", ".png", ".webp", ".gif")):
            # 取得相對路徑（例如 "images/episode02/episode02_001_你邀燈一起組樂團?.jpg"）
            relative_path = os.path.relpath(os.path.join(root, file), image_dir)
            image_data.append(relative_path.replace("\\", "/"))  # 確保相容性


image_data.sort()
# 將結果存成 JSON 檔案
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(image_data, f, ensure_ascii=False, indent=4)

print(f"已生成 {output_file}")

幀截圖

In [21]:
import os
from PIL import Image

def capture_frame_at_frame_number(video_path, output_folder, image_format, n_frame):
    os.makedirs(output_folder, exist_ok=True)
    video = load_video(video_path)

    try:
        n_frame = int(n_frame)
        # 計算對應秒數
        time_in_seconds = float(n_frame) / 23.98

        for offset in range(-15, 16):  # 包含 -15 到 +15
            target_frame = n_frame + offset
            target_time = float(target_frame) / 23.98

            frame = video.get_frame(target_time)

            frame_filename = f"{int(target_frame):04d}.{image_format}"
            frame_path = os.path.join(output_folder, frame_filename)

            Image.fromarray(frame).save(frame_path)
            #print(f"已儲存: {frame_path}")
    except ValueError:
        print("請輸入有效的數字幀數。")


def clear_jpg_files(folder_path):
    # 確保資料夾存在
    if os.path.exists(folder_path):
        # 遍歷資料夾中的所有檔案
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)
            # 檢查檔案是否是 .jpg 格式
            if os.path.isfile(file_path) and filename.lower().endswith('.jpg'):
                try:
                    os.remove(file_path)  # 刪除檔案
                    #print(f"已刪除檔案: {file_path}")
                except Exception as e:
                    print(f"無法刪除 {file_path}: {e}")
    else:
        print(f"資料夾 {folder_path} 不存在。")

In [ ]:
video_path = "/content/drive/MyDrive/SAS/ave_mujica_video/[ANi] BanG Dream! Ave Mujica - 07 [1080P][Baha][WEB-DL][AAC AVC][CHT].mp4"
output_path = "/content/drive/MyDrive/SAS/screenshot"
image_format = "jpg"

while True:
    try:
        n_frame = int(input("請輸入幀數 (輸入 -1 結束): "))
        if n_frame == -1:
            print("結束程序")
            break

        #clear_jpg_files(output_path)
        capture_frame_at_frame_number(video_path, output_path, image_format, n_frame)

    except ValueError:
        print("請輸入有效的數字！")

修改檔名

In [ ]:
import os
drive.mount('/content/drive')

def batch_rename(target_path):
    # 检查目标路径是否存在
    if not os.path.exists(target_path):
        print(f"错误：路径 {target_path} 不存在")
        return

    files = os.listdir(target_path)

    for filename in files:
        if ' 的副本' in filename:
            new_name = filename.replace(' 的副本', '')
            src = os.path.join(target_path, filename)
            dst = os.path.join(target_path, new_name)

            if os.path.exists(dst):
                print(f"警告：{dst} 已存在，跳過修改")
                continue

            os.rename(src, dst)
            print(f"已修改：{src} -> {dst}")

if __name__ == '__main__':
    target_dir = '/content/drive/MyDrive/SAS/ave_mujica_video'  # 指定目标目录

    print("=== 批量修改檔名工具 ===")
    print(f"目標目錄：{target_dir}\n")

    preview = [f for f in os.listdir(target_dir) if ' 的副本' in f]
    print("將修改以下檔案：")
    for f in preview:
        print(f"• {f}")

    confirm = input("\n確定要執行修改嗎？(y/n): ").lower()
    if confirm == 'y':
        batch_rename(target_dir)
        print("\n修改完成！")
    else:
        print("\n已取消操作")